# Gradient decent

## Exact Gradient Computation

Given a function f, sometimes we can compute its exact gradient at any x if f's derivative is easy to compute. For example, let $f(x)=2x^2-3x+\ln x$, where $x>0$. Please compute the derivative of f and report its gradient at $x=2$.

Your answer: 5.5

## Numerical Gradient Computation [5 pts]

Instead of computing the derivative of a function, we can also estimate the gradient numerically with various methods. These methods are essential, especially when a callable function is not exposed due to privacy reasons, or it is hard to differentiate analytically. 

To numerically compute the gradient, the simple way is to follow Newton's difference quotient: $f'(x)=\lim_{h\to 0}{f(x+h)-f(x)\over h}$. Another two-point formula is to compute the slope through the points $(x-h,f(x-h))$ and $(x+h,f(x+h))$. Let us reuse the example function $f(x)=2x^2-3x+\ln x$ and test the precision of these two approaches. Define the function in the next cell, and try to compute its gradient via both methods at $x=2$. Range h value in [0.1,0.01,0.001,0.0001] and report all gradients calculated. Which method is more accurate, and why does it work better?

In [144]:
import math

def f(x):
    # Your code here
    return 2*x**2 - 3*x + math.log(x)

x = 2
hs = [0.1, 0.01, 0.001, 0.0001]

In [145]:
# Compute gradient using the first method (Newton's difference quotient)
for h in hs:
    gradient1 = (f(x + h) - f(x)) / h
    print("Gradient using Newton's difference quotient with h={}: {:.5f}".format(h, gradient1))

# Compute gradient using the second method

Gradient using Newton's difference quotient with h=0.1: 5.68790
Gradient using Newton's difference quotient with h=0.01: 5.51875
Gradient using Newton's difference quotient with h=0.001: 5.50188
Gradient using Newton's difference quotient with h=0.0001: 5.50019


In [146]:
# Compute gradient using the second method 
for h in hs:
    gradient2 = (f(x + h) - f(x - h)) / (2 * h)
    print("Gradient using two-point formula with h={}: {:.5f}".format(h, gradient2))

Gradient using two-point formula with h=0.1: 5.50042
Gradient using two-point formula with h=0.01: 5.50000
Gradient using two-point formula with h=0.001: 5.50000
Gradient using two-point formula with h=0.0001: 5.50000


#### Remark: You may find the gradient more accurate when using a smaller h value. However, this is not always the case. Due to the finite precision of the floating-point, rounding errors always exist and can dominate the computation when the h value is too small. Run the following two cells to observe such scenarios.

In [147]:
eps = 1e-15
print((f(2+eps)-f(2-eps))/2/eps)

5.551115123125783


In [148]:
eps = 1e-20
print((f(2+eps)-f(2-eps))/2/eps)

0.0


## Logistic regression

Logistic regression is a classification tool that models the probability of an event taking place by having the log odds for the event be a linear combination of one or more independent variables. Specifically, let $\vec{x}=<x_1,\dots ,x_m>$ be an m dimensional vector of independent variables (features), and $y$ be the corresponding binary dependent variable (label). The probability of having $y=1$ is modeled as $$P_y={1\over 1+e^{-(b_0+b_1\cdot x_1+\dots +b_m\cdot x_m)}}={1\over 1+e^{-(b_0+\vec{b}_{1:m}\cdot\vec{x})}}$$

Given a set of data points $<\vec{x}_k,y_k>$ with $k\in [1,n]$, how can we fit the model with these data, i.e., how to choose the best $\vec{b}=b_0,b_1\cdots,b_m$?

One way is to write out the likelihood $$\prod_{k:y_k=1}P_{y_k}\prod_{k:y_k=0}(1-P_{y_k})$$ and find $b_0,b_1\cdots,b_m$ that maximize its logarithm, $$l=\sum_{k:y_k=1}\ln(P_{y_k})+\sum_{k:y_k=0}\ln(1-P_{y_k})$$

In contrast to computing the closed form gradient of a Least-squares loss in a linear model (chapter 5 of MML book), doing the same for logistic regression is not possible. Gradient descent can be used to optimize such function $l$, and we will implement it step-by-step. First, write a function log_likelihood in the next cell that computes the log-likelihood given data points and $\vec{b}$. [5 pts]

In [149]:
import numpy as np
import sklearn

In [168]:
def log_likelihood(X,y,b):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    b: one dimension numpy data array of length m+1
    
    Return the log likelihood.
    """
    n = X.shape[0]
    X_b = np.insert(X, 0, 1, axis=1)
    z = np.dot(X_b, b)
    log_likelihood = np.sum(y*z - np.log(1 + np.exp(z)))
    return log_likelihood

### Test your log_likelihood function with a small example below.

In [169]:
X=np.array([[0.1],[0.5],[1.]])
y=np.array([0,0,1])
b=np.array([0.,1.])
# Your answer should be around -2.03
log_likelihood(X,y,b)

-2.0317353317719005

Now that we have a function to maximize, the next step is to compute the current gradient for parameter $\vec{b}$. Use the method with Newton's difference quotient, and set $h=0.0001$. Implement the function compute_gradient in the next cell that returns the gradients of $\vec{b}$. [7 pts]

In [170]:
def compute_gradient(X,y,b):
# The inputs are the same as the ones of log_likelihood
    n, m = X.shape
    X_b = np.insert(X, 0, 1, axis=1)
    z = np.dot(X_b, b)
    p = 1 / (1 + np.exp(-z))
    h = 0.0001
    
    # Compute the gradient using Newton's difference quotient
    gradient = np.zeros((m+1,))
    for i in range(m+1):
        delta_b = np.zeros((m+1,))
        delta_b[i] = h
        b_plus = b + delta_b
        b_minus = b - delta_b
        gradient[i] = (log_likelihood(X, y, b_plus) - log_likelihood(X, y, b_minus)) / (2*h)
    
    return gradient

In [171]:
# Test your function here, preserve the output
compute_gradient(X,y,b)

array([-0.8784971 , -0.09478616])

Once we know how to compute the gradients, we can optimize the objective, which is log-likelihood in our case, using gradient descent. It iteratively changes the parameters in a small "step" towards the gradient direction, i.e., the direction where the objective increases at the fastest pace. Formally, denote the calculated gradients as $\Delta (\vec{b})$, we can update our parameters via $\vec{b}=\vec{b}+\gamma \cdot \Delta (\vec{b})$, where $\gamma$ is the size of the "step". Repeat this process until the objective stop improving or a pre-set max number of iterations is reached. **Note in practice, the value of gradient changes over iterations and can be very large/small, so you should normalize the gradient vector every iteration, i.e., scale it to $\Delta (\vec{b})\over ||\Delta (\vec{b})||_2$, before using it to compute the new $\vec{b}$. Therefore, the update rule for parameters becomes $\vec{b}=\vec{b}+\gamma \cdot {\Delta (\vec{b})\over ||\Delta (\vec{b})||_2}$**.

Implement the gradient_descent function below. [7 pts]

In [174]:
import numpy as np

def gradient_descent(X, y, initial_b, step_size, max_iteration):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    initial_b: one dimension numpy data array of length m+1
    step_size: scalar, the size of one step update
    max_iteration: scalar, the max number of iterations
    Return the updated coefficient vector b.
    """
    b = np.copy(initial_b)
    
    # Iterate for a maximum of max_iteration times
    for i in range(max_iteration):
        # Calculate the gradient of the log likelihood function
        X_b = np.insert(X, 0, 1, axis=1)
        z = np.dot(X_b, b)
        p = 1 / (1 + np.exp(-z))
        gradient = np.dot(X_b.T, y - p)
        
        # Normalize the gradient vector
        norm_gradient = np.linalg.norm(gradient)
        if norm_gradient != 0:
            normalized_gradient = gradient / norm_gradient
        else:
            normalized_gradient = gradient
        
        # Update the parameter vector
        b += step_size * normalized_gradient
        
    return b

Test the function with the previous example again. Print for each sample from X, based on your model, the probability of having label=1.

In [175]:
optimized_b = gradient_descent(X, y, b, 0.1, 1000)

# compute and print the probability for each row in X below using optimized_b

X_b = np.insert(X, 0, 1, axis=1)
z = np.dot(X_b, optimized_b)
p = 1 / (1 + np.exp(-z))

# Print the probabilities
for i in range(len(X)):
    print(f"Sample {i}: {p[i]}")

Sample 0: 2.1829364876087817e-23
Sample 1: 2.072396465550037e-09
Sample 2: 0.9999999983716885


Next, we apply the implemented logistic regression model to a real dataset. The dataset is a trimmed breast-cancer-Wisconsin dataset from UCI machine learning Repository. Only 100 data points are offered in the training set to make sure the computation can be finished swiftly, no matter how you implement the optimizer. The training dataset is loaded in the next cell, and the vector $\vec{b}$ is also randomly initialized. 

Fit three models with the training set using different step size ranging in [0.01,0.05,0.1] and set the max number of iterations as 10000. How do the final log-likelihood value and the number of iterations change with different step sizes? [7 pts]

In [184]:
f = open("breast-cancer-wisconsin.data","r")
X_train = []
y_train = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_train.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_train.append(tmp)
X_train = np.array(X_train)
y_train = np.array(y_train)
random_b = np.random.uniform(0,1,size=(10))

In [186]:
# Fit three models with different step size, report the final log-likelihood, 
# number of iterations and the final coefficent vector b.
step_sizes = [0.01, 0.05, 0.1]

# Loop through step sizes and fit models
for step_size in step_sizes:
    # Fit model
    b = gradient_descent(X_train[:100], y_train[:100], random_b, step_size, 10000)
    
    # Compute final log-likelihood and number of iterations
    log_likelihood_val = log_likelihood(X_train[:100], y_train[:100], b)
    num_iterations = 10000
    
    # Report results
    print(f"Step size: {step_size}")
    print(f"Final log-likelihood: {log_likelihood_val}")
    print(f"Number of iterations: {num_iterations}")
    print(f"Final coefficient vector b: {b}")
    print()

Step size: 0.01
Final log-likelihood: -7.184454007465381
Number of iterations: 10000
Final coefficient vector b: [-17.22481314   1.1086023   -1.84215934   1.18241234   1.08287278
   0.60787653  -0.07158884   1.04569994   1.28056153   1.75845375]

Step size: 0.05
Final log-likelihood: -7.271870765016004
Number of iterations: 10000
Final coefficient vector b: [-17.51146899   1.14207817  -1.87202347   1.22138382   1.10061113
   0.63603338  -0.06535613   1.07491333   1.31292164   1.79771631]

Step size: 0.1
Final log-likelihood: -7.626185192679759
Number of iterations: 10000
Final coefficient vector b: [-17.98610595   1.18575796  -1.91381785   1.2693273    1.1267102
   0.67422772  -0.06034418   1.11129388   1.35568356   1.85493121]



Finally, load the test dataset, and predict for each sample in the test set what labels it should have using the model obtained. Compare your results with the ground truth labels, and report the accuracy rate. [4 pts]

In [191]:
def predict(X, b):
    X_b = np.insert(X, 0, 1, axis=1)
    z = np.dot(X_b, b)
    p = 1 / (1 + np.exp(-z))
    y_pred = np.round(p)
    return y_pred

f = open("test_data.txt","r")
X_test = []
y_test = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_test.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_test.append(tmp)

In [192]:
# Predict based on your models and report the accuracy
for step_size in [0.01, 0.05, 0.1]:
    b = gradient_descent(X_train, y_train, random_b, step_size, 10000)
    y_pred = predict(X_test, b)
    accuracy = np.mean(y_pred == y_test)
    print("Step size:", step_size)
    print("Accuracy:", accuracy)

Step size: 0.01
Accuracy: 0.9
Step size: 0.05
Accuracy: 0.9
Step size: 0.1
Accuracy: 0.9
